In [1]:
"""
from datetime import date, timedelta

start_date = date.today() - timedelta(30)

print(start_date)

"""

StatementMeta(, abad638a-cfc2-4f8f-a951-dc4ee46f4648, 3, Finished, Available, Finished)

2024-08-10


In [3]:
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

# Read the JSON data into a Spark DataFrame
df = spark.read.option("multiline","true").json(f"Files/{start_date}_earthquake_data.json")

# Select specific fields and rename them for clarity, extracting coordinates and properties from nested fields
df = df.select(
    'id',
    col('geometry.coordinates').getItem(0).alias('longitude'),
    col('geometry.coordinates').getItem(1).alias('latitude'),
    col('geometry.coordinates').getItem(2).alias('elevation'),
    col('properties.title').alias('title'),
    col('properties.place').alias('place_description'),
    col('properties.sig').alias('significance'),
    col('properties.mag').alias('magnitude'),
    col('properties.time').alias('time'),
    col('properties.updated').alias('updated')

)

# Convert 'time' and 'updated' columns from milliseconds to seconds and cast them to TimestampType
df = df.withColumn('time', col('time')/1000).\
        withColumn('updated', col('updated')/1000).\
        withColumn('time', col('time').cast(TimestampType())).\
        withColumn('updated', col('updated').cast(TimestampType()))



# Append the processed data into silver table 'earthquake_events_silver'
df.write.mode('overwrite').saveAsTable('earthquake_events_silver')



StatementMeta(, abad638a-cfc2-4f8f-a951-dc4ee46f4648, 5, Finished, Available, Finished)